In [1]:
#Import the necessary libraries.
import gradio as gr
from transformers import pipeline  


In [ ]:
#Initialize a text summarization pipeline using the T5-small model.
summarizer = pipeline("summarization", model="t5-small")  

Device set to use cpu


In [3]:
#Define a function to summarize text. Create error messages for any errors. 
def summarize_text(input_text, use_file=False, file_path=None):
    if use_file:  
        if file_path:
            try:
                with open(file_path, "r", encoding="utf-8") as file:  
                    text_to_summarize = file.read()
            except FileNotFoundError:
                return "Error: File not found."
            except Exception as e:  
                return f"Error reading file: {e}"
        else:
            return "Error: Please provide a file path."
    else:
        text_to_summarize = input_text  

    try:
        summary = summarizer(text_to_summarize, max_length=200, min_length=30, do_sample=False) 
        return summary[0]["summary_text"]
    except Exception as e:
        return f"Error during summarization: {e}"


In [ ]:
#Define a function to save the summary to a file. Create error messages for any errors.
def save_summary_to_file(summary_text, output_filename="summary.txt"):
    try:
        with open(output_filename, "w", encoding="utf-8") as outfile:
            outfile.write(summary_text)
        return f"Summary saved to {output_filename}"
    except Exception as e:
        return f"Error saving summary: {e}"



In [10]:
#Use Gradio to create a simple UI for the text summarization model.
with gr.Blocks() as demo:
    with gr.Row():
        text_input = gr.Textbox(label="Enter text to summarize", lines=5)
        file_input = gr.File(label="Upload .txt file", file_types=[".txt"])
        file_path_input = gr.Textbox(label="Or, enter the file path")

    with gr.Row():
        output_text = gr.Textbox(label="Summarized text", lines=5)

    with gr.Row():
        save_button = gr.Button("Save Summary to File")
        save_message = gr.Textbox(label="Save Status", value="")
        btn = gr.Button("Summarize")

    def process_inputs(text_input, file_input, file_path_input):
        if file_input:  # If a file was uploaded
            try:
                with open(file_input.name, 'r', encoding='utf-8') as f:
                    file_content = f.read()
                summary = summarize_text(file_content, use_file=True)
                return summary 
            except Exception as e:
                return f"Error processing uploaded file: {e}"
        elif file_path_input:
            summary = summarize_text("", use_file=True, file_path=file_path_input)
            return summary
        else:
            summary = summarize_text(text_input)
            return summary

    def save_summary(summary_text):
        return save_summary_to_file(summary_text)

    btn.click(process_inputs, inputs=[text_input, file_input, file_path_input], outputs=output_text)
    save_button.click(save_summary, inputs=output_text, outputs=save_message)  # Call save function

demo.launch()

        

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Your max_length is set to 200, but your input_length is only 144. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=72)
